Created on: Thu Wed 15 15:40:39 2020
<br>
Group 7
<br>
@author: P.S.B.

In [45]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile
import shutil
import random
import pickle

# Deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

In [46]:
%run g7_functions_for_models.ipynb

# Data preparation


In [56]:
project_path = './../' 
Airliners_path = project_path + 'G7_scrapping/Airliners/data'
new_paths = [project_path + 'Split_Data/Airliners/Train', project_path + 'Split_Data/Airliners/Test']
airbus_planes = ['A320', 'A321', 'A350', 'A330', 'A380']
boeing_planes = ['737', '747', '757', '777']

# nb_types : number of classes to predict
nb_types = len(airbus_planes) + len(boeing_planes)

# Images parameters
size = (224, 224)
greys = False

In [51]:
sep_train_test_airliners(Airliners_path, new_paths, airbus_planes)

In [53]:
data_augmentation(project_path + 'Split_Data/Airliners/Train', nb_img=2)

737
A330
747
A320
A350
777
A380
757
A321


In [73]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(project_path + 'Split_Data/Airliners/Train',
                                                 target_size=size,
                                                 color_mode='rgb',
                                                 batch_size=128,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 23386 images belonging to 9 classes.


In [74]:
train_generator.class_indices

{'737': 0,
 '747': 1,
 '757': 2,
 '777': 3,
 'A320': 4,
 'A321': 5,
 'A330': 6,
 'A350': 7,
 'A380': 8}

In [75]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(project_path + 'Split_Data/Airliners/Test',
                                                 target_size=size,
                                                 color_mode='rgb',
                                                 batch_size=128,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2700 images belonging to 9 classes.


# Simple model

In [116]:
def create_model(input_shape: tuple, nb_layers: int=5, nb_neurons: int=10, kernel: tuple=(3, 3),
                 pool: tuple=(2, 2), nb_classes: int=1):
    
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    for i in range(nb_layers):
        model.add(Conv2D(2**(nb_neurons + i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
                  
        model.add(Conv2D(2**(nb_neurons + i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(MaxPooling2D(pool_size=pool))

    model.add(Flatten())

    model.add(Dense(1024))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(512))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(nb_classes, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model

In [117]:
model = create_model(input_shape=(size[0], size[1], 3), nb_neurons=4)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
batch_normalization_82 (Batc (None, 254, 254, 16)      64        
_________________________________________________________________
re_lu_82 (ReLU)              (None, 254, 254, 16)      0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 252, 252, 16)      2320      
_________________________________________________________________
batch_normalization_83 (Batc (None, 252, 252, 16)      64        
_________________________________________________________________
re_lu_83 (ReLU)              (None, 252, 252, 16)      0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 126, 126, 16)     

In [118]:
batch_size = 32
epochs = 5

# Lancement de l'entraînement
history = model.fit(data_train, ytrain, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(data_test, ytest))

Train on 2403 samples, validate on 794 samples
Epoch 1/5
2403/2403 [==============================] - 310s 129ms/step - loss: 0.7931 - accuracy: 0.5734 - val_loss: 0.6665 - val_accuracy: 0.5957
Epoch 2/5
2403/2403 [==============================] - 299s 124ms/step - loss: 0.6023 - accuracy: 0.6787 - val_loss: 0.6893 - val_accuracy: 0.5592
Epoch 3/5
2403/2403 [==============================] - 293s 122ms/step - loss: 0.5261 - accuracy: 0.7399 - val_loss: 0.7679 - val_accuracy: 0.6222
Epoch 4/5
2403/2403 [==============================] - 292s 122ms/step - loss: 0.6316 - accuracy: 0.6633 - val_loss: 10.0990 - val_accuracy: 0.5151
Epoch 5/5
2403/2403 [==============================] - 293s 122ms/step - loss: 0.5147 - accuracy: 0.7511 - val_loss: 1.2679 - val_accuracy: 0.5781


# Transfer learning

In [76]:
# Get the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(size[0], size[1], 1 if greys else 3))
x = base_model.output

# Add layers
'''x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)'''

x = Flatten()(x)  # vector

# Fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Output layer to predict the class
predictions = Dense(nb_types, activation = 'softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data = test_generator,
                   callbacks=[reduce_lr])

Epoch 1/5
182/182 [==============================] - 4249s 23s/step - loss: 1.1960 - accuracy: 0.5795 - val_loss: 1.4383 - val_accuracy: 0.7278
Epoch 2/5
182/182 [==============================] - 10377s 57s/step - loss: 0.2986 - accuracy: 0.9035 - val_loss: 0.0170 - val_accuracy: 0.7715
Epoch 3/5
 75/182 [===========>..................] - ETA: 36:16 - loss: 0.1071 - accuracy: 0.9699

In [55]:
path_mod = './../Models/'
save_model_classes(path_mod, 'Ext_typ', train_generator, model)

NameError: name 'model' is not defined